# Schreier-Sims for Rubik's Cubes

In [ ]:
import numpy as np, time, random
from sympy.combinatorics import Permutation
from functions import enumerate_cube, human_order, solved_states, compute_schreiers_vector, generating_set, sims_filter
import config

# choose size of Rubik's cube
n = 3

# N: order of the symmetric group (number of enumerated stickers)
if n % 2 == 0:
    N = 6 * n**2
else:
    N = 6 * n**2 - 6

# set of moves that generate the Rubik's cube
generators = enumerate_cube(n, N)

identity_order = range(N)
random_order = random.sample(range(N), N)
human_order = human_order(n)
# choose solving order
solving_order = human_order

# list of group orders of the stabilizer groups
orders_list = []

# list of generating sets of the stabilizer groups
generators_list = []
length_generators_list_before_sims_filter = []
length_generators_list = []

# list of Schreiers vectors for the stabilizer groups
schreiers_vectors_list = []

# dictionary of lengths of moves written as composition of original generators
config.lengths = {Permutation(N-1):0}
for generator in generators:
    config.lengths[generator] = 1

# the algorithm
i = 0

length_generators_list_before_sims_filter.append(len(generators))

temp_generators = sims_filter(generators, N)
length_generators_list.append(len(temp_generators))
generators_list.append(temp_generators)

start_time = time.time()

while len(temp_generators) > 0:
    k = solving_order[i]

    schreiers_vector = compute_schreiers_vector(temp_generators, N, k)
    schreiers_vectors_list.append(schreiers_vector)

    schreiers_vector_non_zero = [element for element in schreiers_vector if element is not None]

    order_orbit = len(schreiers_vector_non_zero)
    orders_list.append(order_orbit)

    new_generators = generating_set(temp_generators, schreiers_vector, schreiers_vector_non_zero, k)
    length_generators_list_before_sims_filter.append(len(new_generators))
    # print(f"Length of new generators before Sims filter: {len(new_generators)}")

    temp_generators = sims_filter(new_generators, N)
    length_generators_list.append(len(temp_generators))
    generators_list.append(temp_generators)
    # print(f"Length of new generators after Sims filter: {len(temp_generators)}")

    i += 1

    elapsed_time = time.time()-start_time
    print(f"Time after iteration {i}: {elapsed_time:.2f} seconds")


# compute visually solved states (useful for membership testing)
solved_states = solved_states(n, N)

## Order of Rubik's cube group

In [ ]:
orders_list = np.array(orders_list, dtype=object)
print(np.prod(orders_list))

## Membership testing and length of solving algorithm

In [ ]:
# choose the element you want to perform the check on

# some examples of moves:

for generator in generators:
    move = Permutation(generator)
    print(generator)
# move = Permutation(N-1)(0,10,20)(6,12,40) # double corner twist on 3x3
# move = Permutation(N-1)(51,63)(0,40,20)(32,52,72) # double corner twist and enter swap on 4x4

# the algorithm

    contained = False
    length_solving_algorithm = 0

    for solved_state in solved_states:
        g = move * solved_state
        length_solving_algorithm = 0

        i = 0
        while i < len(schreiers_vectors_list):
            k = solving_order[i]

            orbit_set = schreiers_vectors_list[i]
            h = g(k)
            u = orbit_set[h]
            if u is None:
                break
            else:
                g = g * u**-1
                length_solving_algorithm += config.lengths[u]
            i += 1

        if g == Permutation(N-1):
            contained = True
            print(length_solving_algorithm)
            break

    print(contained)